# Analyse des sentiments d'une interview en français

Analyse des sentiments pour déterminer la Polarité et la Subjectivité des réponses.

- Une polarité supérieure à 0 est positive
- Une polarité inférieure à 0 est négative
- Une polarité égale à 0 est neutre

Il est tout à fait possible de modifier les intervalles de valeurs de la polarité pour définir un seuil de positivité / négativité. **Nous prenons ici des seuils de 0.1 et -0.1**
  
L'analyse calcule également la Subjectivité comprise entre 0 et 1. Plus la valeur approche 1 est plus la subjectivité est grande.

L'analyse des sentiments est réalisées sur un texte nettoyé

Le script utilise le package TextBlob https://textblob.readthedocs.io/en/dev/

Le corpus de texte étant en français, la version textblob_fr est nécessaire https://github.com/sloria/textblob-fr

**Attention**, la librairie calcul la polarité et subjectivité moyenne à partir de chaque mot. Pour cela textblob fait appel à un dictionnaire de mots dont le score a été attribué à la main. Il s'agit de la librairie **pattern** qui récupère chaque score de mot à partir de **sentiwordnet**

La librairie textblob fonctionne correctement dans la plupart du temps, elle peut cependant générer des erreurs dans certains cas.

# Librairies

In [ ]:
# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Traitement du langage naturel (NLP)
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer   
## Télécharger si besoin
#nltk.download('punkt')
#nltk.download('stopwords')
import string

from textblob import TextBlob # Analyse des sentiments
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

# Dataset

Le dataset est composé de quetions et réponses à l'interview. Chaque réponse est découpée en phrase afin d'analyser les sentiments par phrase.

In [ ]:
# Choisir le fichier source
data = pd.read_csv('data_sentences.csv', sep =',',
                  dtype = {'questions' : str,
                           'réponses' : str}
                  )
data.rename({'réponses' : 'texte'}, axis = 1, inplace = True)

In [ ]:
print(data.info())
data.head()

# Data processing 

Consiste à retraiter le texte de chaque réponse pour conserver les mots contenant de l'information. 

In [ ]:
# Tokenizer
def tokenizer_fct(sentence) :
    # print(sentence)
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean, language='french')
    return word_tokens

# Stop words
stop_w = list(set(stopwords.words('french'))) + ['[', ']', ',', '.', ':', '?', '(', ')']

def stop_word_filter_fct(list_words) :
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

# Lower case et alpha
def lower_start_fct(list_words) :
    lw = [w.lower() for w in list_words if (not w.startswith("@")) 
                                       and (not w.startswith("http"))]
    return lw

# Lemmatizer (base d'un mot)
def lemma_fct(list_words) :
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

# Fonction de préparation du texte sans lemmatization
def transform_bow_fct(desc_text) :
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    # lem_w = lemma_fct(lw)    
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Fonction de préparation du texte pour le bag of words avec lemmatization
def transform_bow_lem_fct(desc_text) :
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)    
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text

In [ ]:
data['cleaned_texte'] = data['texte'].apply(lambda x : transform_bow_lem_fct(x)).copy()
data

# Analyse des sentiments

Cette partie utilise textblob-fr pour l'analyse des sentiments à partir du texte nettoyé.

Le sript génnère les colonnes polarity, subjectivité et Sentiment

## Sentiment Analyzer

In [ ]:
# Utilisation de la classe Blobber
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
# Calcule de la polarité et subjectivité
data['polarity'] = data['cleaned_texte'].apply(lambda x: tb(x).sentiment[0])
data['subjectivity'] = data['cleaned_texte'].apply(lambda x: tb(x).sentiment[1])
# Définir les étiquettes de sentiment en fonction de la polarité
data['Sentiment'] = data['polarity'].apply(lambda x: 'Positif' if x > 0.1 else ('Negatif' if x < -0.1 else 'Neutre'))

# Corrige la subjectivité lorsqu'elle sort de l'intervalle 0 - 1 (erreur de la librairie)
data.loc[data['subjectivity'] < 0, 'subjectivity'] = 0
data.loc[data['subjectivity'] > 1, 'subjectivity'] = 1

In [ ]:
data.head(15)

## Analyse globale

In [ ]:
# Affiche les percentiles
data.describe(percentiles=[.1, .2, .3, .4, .5,
                          .6, .7, .8, .9])

In [ ]:
# Phrases les plus polarisantes
polarisation = data.sort_values(by='polarity', ascending = False)

In [ ]:
top_positif = polarisation.head(3)
top_positif

In [ ]:
top_negatif = polarisation.tail(3).sort_values(by='polarity', ascending = True)
top_negatif

In [ ]:
# Compter le nombre de chaque sentiment (Positif, Negatif, Neutre) pour faire un diagramme
sentiment_counts = data['Sentiment'].value_counts()

# Récupère les labels
labels = sentiment_counts.index.tolist()
values = sentiment_counts.values.tolist()

colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']

# Trace le diagramme circulaire
plt.pie(values, labels=labels,
        colors = colors, autopct='%1.1f%%')
plt.title("Répartition des sentiments")
plt.show()

In [ ]:
# Boxplot pour visualiser les valeurs atypiques
data[['polarity', 'subjectivity']].plot(kind='box', vert=False)
plt.title('Boxplot de la polarité et de la subjectivité')
plt.xlabel('Valeur')
plt.ylabel('Variables')


plt.show()

In [ ]:
# Scatter plot
fig = px.scatter(data,x='polarity',y='subjectivity')
fig.show()

## Analyse par question

In [ ]:
# Regroupe les sentiments par questions
df_agg = data.groupby(['questions', 'Sentiment']).size().unstack(fill_value=0)
df_agg.reset_index(inplace=True)
df_agg

In [ ]:
# Agrégation avec des statistiques supplémentaires
data_by_questions = data.groupby('questions').agg({
    'polarity': ['min', 'max', 'mean', 'median'],
    'subjectivity': ['min', 'max', 'mean', 'median']  
})

# Renomme les colonnes
data_by_questions.columns = ['min_polarity', 'max_polarity', 'mean_polarity', 'median_polarity',
                              'min_subjectivity', 'max_subjectivity', 'mean_subjectivity', 'median_subjectivity']

data_by_questions = data_by_questions.reset_index()
data_by_questions.head(15)

In [ ]:
# Agrégation par question pour traiter chaque question individuellement
questions = data['questions'].unique()

for question in questions:
    # Sélectionne les phrases et sentiments pour chaque question
    subset = data[data['questions'] == question]
    
    # Compte le nombre de sentiments pour la question
    sentiment_counts = subset['Sentiment'].value_counts()
    
    # Labels
    labels = sentiment_counts.index.tolist()
    values = sentiment_counts.values.tolist()
    
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    
    # Trace le diagramme circulaire de la question
    plt.figure()  
    plt.pie(values, labels=labels, colors=colors, autopct=lambda pct: f"{pct:.1f}%\n({int(pct/100*sum(values))})")
    
    # Titre avec la question
    plt.title(f"Répartition des sentiments - {question}")
    
    plt.show()


In [ ]:
# Agrégation par question pour traiter chaque question individuellement
questions = data['questions'].unique()

for question in questions:
    # # Sélectionne les phrases et sentiments pour chaque question
    subset = data[data['questions'] == question]
    
    # Tracer le boxplot pour la polarité et la subjectivité pour cette question
    plt.figure()  
    subset[['polarity', 'subjectivity']].plot(kind='box', vert=False)
    
    # Titre des boxplot avec la question
    plt.title(f'Boxplot de la polarité et de la subjectivité - {question}')
    plt.xlabel('Valeur')
    plt.ylabel('Variables')

    plt.show()

In [ ]:
# Agrégation par question pour regrouper les phrases par question
data_by_questions_graphe = data.groupby('questions').agg(list).reset_index()

# Création des scatter plots par question
for index, row in data_by_questions_graphe.iterrows():
    question = row['questions']
    texts = row['cleaned_texte']
    polarities = row['polarity']
    subjectivities = row['subjectivity']
    
    # Création du DataFrame pour le scatter plot
    scatter_df = pd.DataFrame({
        'cleaned_texte': texts,
        'polarity': polarities,
        'subjectivity': subjectivities
    })
    
    # Création du scatter plot avec Plotly Express
    fig = px.scatter(scatter_df, x='polarity', y='subjectivity',
                     labels={'polarity': 'Polarity', 'subjectivity': 'Subjectivity'},
                     title=f'Scatter Plot for Question: {question}')
    
    fig.show()